
## Empacar y desempacar argumentos


Cuando en **Python** creamos una función que acepta un número arbitrario de argumentos estamos utilizando una habilidad del lenguaje que es el "empaquetamiento" y "desempaquetamiento" automático de variables.

Al definir un número variable de argumentos de la forma:
```python
def f(*v):
...
```

y luego utilizarla en alguna de las formas:

```python
f(1)
f(1,'hola')
f(a,2,3.5, 'hola')
```

**Python** automáticamente convierte los argumentos en una única tupla:
```python
f(1)                 --> v = (1,)
f(1,'hola')          --> v = (1,'hola')
f(a,2,3.5, 'hola')   --> v = (a,2,3.5,'hola')
```

Análogamente, cuando utilizamos funciones podemos desempacar múltiples valores en los argumentos de llamada a las funciones.

Si definimos una función que recibe un número determinado de argumentos
```python
def g(a, b, c):
...
```
y definimos una lista (o tupla)
```python
t1 = [a1, b1, c1]
```
podemos realizar la llamada a la función utilizando la notación "asterisco" o "estrella"
```python
g(*t1)             -->  g(a1, b1, c1)
```

Esta notación no se puede utilizar en cualquier contexto. Por ejemplo, es un error tratar de hacer

```python
t2 = *t1
```

pero en el contexto de funciones podemos "desempacarlos" para convertirlos en varios argumentos que acepta la función usando la expresión con asterisco. Veamos lo que esto quiere decir con la función `caida_libre()` definida anteriormente

In [1]:
def caida_libre(t, h0, v0 = 0., g=9.8):
  """Devuelve la velocidad y la posición de una partícula en
  caída libre para condiciones iniciales dadas

  Parameters
  ----------
  t : float
      el tiempo al que queremos realizar el cálculo
  h0: float 
      la altura inicial
  v0: float (opcional)
      la velocidad inicial (default = 0.0)
   g: float (opcional)
      valor de la aceleración de la gravedad (default = 9.8)

  Returns
  -------
  (v,h):  tuple of floats
       v= v0 - g*t
       h= h0 - v0*t -g*t^2/2
  
  """
  v = v0 - g*t
  h = h0 - v0*t - g*t**2/2.
  return v,h


In [2]:
datos = (5.4, 1000., 0.)        # Una lista (tuple en realidad)
# print (caida_libre(datos[0], datos[1], datos[2]))
print (caida_libre(*datos))

(-52.92000000000001, 857.116)


En la llamada a la función, la expresión `*datos` le indica al intérprete Python que la secuencia (tuple) debe convertirse en una sucesión de argumentos, que es lo que acepta la función. 

Similarmente, para desempacar un diccionario usamos la notación `**diccionario`:

In [3]:
# diccionario, caída libre en la luna
otros_datos = {'t':5.4, 'h0': 1000., "g" : 1.625} 
v, h = caida_libre(**otros_datos)
print ('v={}, h={}'.format(v,h))


v=-8.775, h=976.3075


En resumen:

* la notación `(*datos)` convierte la tuple (o lista) en los tres argumentos que acepta la función caída libre. Los siguientes llamados son equivalentes:
```python
caida_libre(*datos)
caida_libre(datos[0], datos[1], datos[2])
caida_libre(5.4, 1000., 0.)
```
* la notación `(**otros_datos)` desempaca el diccionario en pares `clave=valor`, siendo equivalentes los dos llamados:
```python
caida_libre(**otros_datos)
caida_libre(t=5.4, h0=1000., g=0.2)
```


##  Funciones que devuelven funciones

Las funciones pueden ser pasadas como argumento y pueden ser retornadas por una función, como cualquier otro objeto (números, listas, tuples, cadenas de caracteres, diccionarios, etc). Veamos un ejemplo simple de funciones que devuelven una función:

In [4]:
def crear_potencia(n):
  "Devuelve la función x^n"
  def potencia(x):
    "potencia {}-esima de x".format(n)
    return x**n
  return potencia

In [5]:
f = crear_potencia(3)
print(f)
cubos = [f(j) for j in range(5)]

<function crear_potencia.<locals>.potencia at 0x7ff1734dac00>


In [6]:
cubos

[0, 1, 8, 27, 64]

In [7]:
help(f)

Help on function potencia in module __main__:

potencia(x)



In [8]:
help(crear_potencia)

Help on function crear_potencia in module __main__:

crear_potencia(n)
    Devuelve la función x^n



## Ejemplo: Polinomio interpolador

Veamos ahora una función que retorna una función. Supongamos que tenemos una tabla de puntos `(x,y)` por los que pasan nuestros datos y queremos interpolar los datos con un polinomio.

Sabemos que dados `N` puntos, hay un único polinomio de grado `N` que pasa por todos los puntos. En este ejemplo utilizamos la fórmula de Lagrange para obtenerlo.

In [9]:
# %load scripts/ejemplo_05_2.py
def polinomio_interp(x, y):
  """Devuelve el polinomio interpolador que pasa por los puntos (x_i, y_i)

    Warning: La implementación es numéricamente inestable. Funciona para algunos puntos (menor a 20)

  Keyword Arguments:
  x -- Lista con los valores de x
  y -- Lista con los valores de y
  """

  M = len(x)

  def polin(xx):
    """Evalúa el polinomio interpolador de Lagrange"""
    P = 0

    for j in range(M):
      pt = y[j]
      for k in range(M):
        if k == j:
          continue
        fac = x[j] - x[k]
        pt *= (xx - x[k]) / fac
      P += pt
    return P

  return polin


Lo que obtenemos al llamar a esta función es una función

In [10]:
f = polinomio_interp([0,1], [0,2])

In [11]:
f

<function __main__.polinomio_interp.<locals>.polin(xx)>

In [12]:
help(f)

Help on function polin in module __main__:

polin(xx)
    Evalúa el polinomio interpolador de Lagrange



In [13]:
f(3.4)

6.8

Este es el resultado esperado porque queremos el polinomio que pasa por dos puntos (una recta), y en este caso es la recta $y = 2x$. Veamos cómo usarlo, en forma más general:

In [14]:
# %load scripts/ejemplo_05_3
from ejemplo_05_2 import polinomio_interp

xmax = 5
step = 0.2
N = int(5 / step)

x2, y2 = [1, 2, 3], [1, 4, 9]   # x^2
f2 = polinomio_interp(x2, y2)

x3, y3 = [0, 1, 2, 3], [0, 2, 16, 54]  # 2 x^3
f3 = polinomio_interp(x3, y3)

print('\n x   f2(x)   f3(x)\n' + 18 * '-')
for j in range(N):
  x = step * j
  print(f'{x:.1f}  {f2(x):5.2f}  {f3(x):6.2f}')



 x   f2(x)   f3(x)
------------------
0.0   0.00    0.00
0.2   0.04    0.02
0.4   0.16    0.13
0.6   0.36    0.43
0.8   0.64    1.02
1.0   1.00    2.00
1.2   1.44    3.46
1.4   1.96    5.49
1.6   2.56    8.19
1.8   3.24   11.66
2.0   4.00   16.00
2.2   4.84   21.30
2.4   5.76   27.65
2.6   6.76   35.15
2.8   7.84   43.90
3.0   9.00   54.00
3.2  10.24   65.54
3.4  11.56   78.61
3.6  12.96   93.31
3.8  14.44  109.74
4.0  16.00  128.00
4.2  17.64  148.18
4.4  19.36  170.37
4.6  21.16  194.67
4.8  23.04  221.18


-----

## Ejercicios 04 (c)

4. Escriba una función `crear_sen(A, w)` que acepte dos números reales $A, w$ como argumentos y devuelva la función `f(x)`.

  Al evaluar la función `f` en un dado valor $x$ debe dar el resultado: $f(x) = A \sin(w x)$ tal que se pueda utilizar de la siguiente manera:
  
  ```python
  from math import pi
  f = crear_sen(3, w)
  f(2*pi/2)    # Debería imprimir el resultado de 3*sin(pi) aprox. cero
  ```

-----


## Funciones que toman como argumento una función

In [15]:
def aplicar_fun(f, L):
  """Aplica la función f a cada elemento del iterable L 
  devuelve una lista con los resultados.
  
  IMPORTANTE: Notar que no se realiza ninguna comprobación de validez
  """
  return [f(x) for x in L]

In [16]:
import math as m
Lista = list(range(1,10))
t = aplicar_fun(m.sin, Lista)

In [17]:
t

[0.8414709848078965,
 0.9092974268256817,
 0.1411200080598672,
 -0.7568024953079282,
 -0.9589242746631385,
 -0.27941549819892586,
 0.6569865987187891,
 0.9893582466233818,
 0.4121184852417566]

Notar que definimos la función `aplicar_fun()` que recibe una función y una secuencia, pero no necesariamente una lista, por lo que podemos aplicarla directamente a `range`:

In [18]:
aplicar_fun(crear_potencia(3), range(5)) 

[0, 1, 8, 27, 64]

Además, debido a su definición, el primer argumento de la función `aplicar_fun()` no está restringida a funciones numéricas pero al usarla tenemos que asegurar que la función y el iterable (lista) que pasamos como argumentos son compatibles.

Veamos otro ejemplo:

In [19]:
s = ['hola', 'chau']
print(aplicar_fun(str.upper, s))

['HOLA', 'CHAU']


donde `str.upper` es una función definida en **Python**, que convierte a mayúsculas el string dado `str.upper('hola') = 'HOLA'`.


## Aplicacion 1: Ordenamiento de listas

Consideremos el problema del ordenamiento de una lista de strings. Como vemos el resultado usual no es necesariamente el deseado

In [20]:
s1 = ['Estudiantes', 'caballeros', 'Python', 'Curso', 'pc', 'aereo']
print(s1)
print(sorted(s1))

['Estudiantes', 'caballeros', 'Python', 'Curso', 'pc', 'aereo']
['Curso', 'Estudiantes', 'Python', 'aereo', 'caballeros', 'pc']


Acá `sorted` es una función, similar al método `str.sort()` que mencionamos anteriormente, con la diferencia que devuelve una nueva lista con los elementos ordenados.
Como los elementos son *strings*, la comparación se hace respecto a su posición en el abecedario. En este caso no es lo mismo mayúsculas o minúsculas.

In [21]:
s2 = [s.lower() for s in s1]
print(s2)
print(sorted(s2))

['estudiantes', 'caballeros', 'python', 'curso', 'pc', 'aereo']
['aereo', 'caballeros', 'curso', 'estudiantes', 'pc', 'python']


Posiblemente queremos el orden que obtuvimos en segundo lugar pero con los elementos dados originalmente (con sus mayúsculas y minúsculas originales).
Para poder modificar el modo en que se ordenan los elementos, la función `sorted` (y el método `sort`) tienen el argumento opcional `key` que debe ser una función. Entonces `sort()` y `sorted()` toman una función como argumento.

In [22]:
sorted(s1, key=str.lower)

['aereo', 'caballeros', 'Curso', 'Estudiantes', 'pc', 'Python']

Como vemos, los strings están ordenados adecuadamente. Si queremos ordenarlos por longitud de la palabra

In [23]:
sorted(s1, key=len)

['pc', 'Curso', 'aereo', 'Python', 'caballeros', 'Estudiantes']

Supongamos que queremos ordenarla alfabéticamente por la segunda letra

In [24]:
def segunda(a):
  return a[1]

sorted(s1, key=segunda)

['caballeros', 'pc', 'aereo', 'Estudiantes', 'Curso', 'Python']


## Funciones anónimas

En ocasiones como esta suele ser más rápido (o conveniente) definir la función, que se va a utilizar una única vez, sin darle un nombre. Estas se llaman funciones *lambda*, y el ejemplo anterior se escribiría

In [25]:
sorted(s1, key=lambda a: a[1])

['caballeros', 'pc', 'aereo', 'Estudiantes', 'Curso', 'Python']

Si queremos ordenarla alfabéticamente empezando desde la última letra:

In [26]:
sorted(s1, key=lambda a: str.lower(a[::-1]))

['pc', 'Python', 'aereo', 'Curso', 'Estudiantes', 'caballeros']

Este es un ejemplo de uso de las funciones anónimas `lambda`. La forma general de las funciones `lambda` es:

```python
lambda x,y,z: expresión_de(x,y,z)
```
por ejemplo, para calcular $(n+1) x^n$, hicimos:

```python
lambda x,n: (n+1) * x**n

```


## Ejemplo: Integración numérica

Veamos en más detalle el caso de funciones que reciben como argumento otra función, estudiando un caso usual: una función de integración debe recibir como argumento al menos una función a integrar y los límites de integración:

In [27]:
# %load scripts/05_ejemplo_1.py
def integrate_simps(f, a, b, N=10):
  """Calcula numéricamente la integral de la función en el intervalo dado
  utilizando la regla de Simpson

  Keyword Arguments:
  f -- Función a integrar
  a -- Límite inferior
  b -- Límite superior
  N -- El intervalo se separa en 2*N intervalos
  """
  h = (b - a) / (2 * N)
  I = f(a) - f(b)
  for j in range(1, N + 1):
    x2j = a + 2 * j * h
    x2jm1 = a + (2 * j - 1) * h
    I += 2 * f(x2j) + 4 * f(x2jm1)
  return I * h / 3


En este ejemplo programamos la fórmula de integración de Simpson para obtener la integral de una función `f(x)` provista por el usuario, en un dado intervalo:
$$
\int _{a}^{b}f(x)\,dx\approx \frac{h}{3} \bigg[ f(x_{0}) + 2 \sum_{j=1}^{n/2} f(x_{2j}) + 4 \sum_{j=1}^{n/2} f(x_{2j-1}) - f(x_{n})\bigg]
$$

¿Cómo usamos la función de integración?

In [28]:
def potencia2(x):
  return x**2

integrate_simps(potencia2, 0, 3, 7)

9.0

Acá definimos una función, y se la pasamos como argumento a la función de integración. 


### Uso de funciones anónimas

Veamos como sería el uso de funciones anónimas en este contexto

In [29]:
integrate_simps(lambda x: x**2, 0, 3, 7)

9.0

La notación es un poco más corta, que es cómodo pero no muy relevante para un caso.
Si queremos, por ejemplo, aplicar el integrador a una familia de funciones la notación se simplifica notablemente:

In [30]:
print('Integrales:')
a = 0
b = 3
for n in range(6):
  I = integrate_simps(lambda x: (n + 1) * x**n, a, b, 10)
  print(f'I ( {n+1} x^{n}, {a}, {b} ) = {I:.5f}')


Integrales:
I ( 1 x^0, 0, 3 ) = 3.00000
I ( 2 x^1, 0, 3 ) = 9.00000
I ( 3 x^2, 0, 3 ) = 27.00000
I ( 4 x^3, 0, 3 ) = 81.00000
I ( 5 x^4, 0, 3 ) = 243.00101
I ( 6 x^5, 0, 3 ) = 729.00911



-----

## Ejercicios 04 (d)


5. Escriba una serie de funciones que permitan trabajar con polinomios. Vamos a representar a un polinomio como una lista de números reales, donde cada elemento corresponde a un coeficiente que acompaña una potencia. En cada caso elija los argumentos que considere necesario.

    * Una función que devuelva el orden del polinomio (un número entero)
    * Una función que sume dos polinomios y devuelva un polinomio (objeto del mismo tipo)
    * Una función que multiplique dos polinomios y devuelva el resultado en otro polinomio
    * Una función devuelva la derivada del polinomio (otro polinomio).
    * Una función que acepte el polinomio y devuelva la función correspondiente.
 
6. Escriba una función `direccion_media(ang1, ang2, ...)` cuyos argumentos son direcciones en el plano, expresadas por los ángulos en grados a partir de un cierto eje, y calcule la dirección promedio, expresada en ángulos. Pruebe su función con las listas:

   ```python
   a1 = direccion_media(0, 180, 370, 10)
   a2 = direccion_media(30, 0, 80, 180)
   a3 = direccion_media(80, 180, 540, 280)
   ```

7. Las funciones de Bessel de orden $n$ cumplen las relaciones de recurrencia
   $$
    J_{n -1}(x)- \frac{2n }{x}\, J_{n }(x) + J_{n +1}(x) = 0 
   $$
   $$
   J^{2}_{0}(x) + \sum_{n=1}^{\infty} 2 J^{2}_{n}(x) = 1
   $$

   Para calcular la función de Bessel de orden $N$, se empieza con un valor de $M \gg N$, y utilizando los valores iniciales $J_{M}=1$, $J_{M+1}=0$ se utiliza la primera relación para calcular todos los valores de $n < M$. Luego, utilizando la segunda relación se normalizan todos los valores.

   --------

   **Nota:** Estas relaciones son válidas si $M \gg x$ (use algún valor estimado, como por ejemplo $M=N+20$).

   --------

   Utilice estas relaciones para calcular $J_{N}(x)$ para $N=3,4,7$ y $x=2.5, 5.7, 10$.
   Para referencia se dan los valores esperados
   $$
   \begin{align}
   J_3( 2.5) =  0.21660\\
   J_4( 2.5) =  0.07378\\
   J_7( 2.5) =  0.00078\\
   J_3( 5.7) =  0.20228\\
   J_4( 5.7) =  0.38659\\
   J_7( 5.7) =  0.10270\\
   J_3(10.0) =  0.05838\\
   J_4(10.0) = -0.21960\\
   J_7(10.0) =  0.21671\\
   \end{align}
   $$

9. Dada una lista de números, vamos a calcular valores relacionados a su estadística.

    - Realizar una función `calc_media(x, que="aritmetica")` que calcule los valores de la media aritmética, la media geométrica o la media armónica dependiendo del valor del argumento `que`. Las medias están dadas por:
    $$
    A(x_1, \ldots, x_n) = \bar{x} = \frac{x_1 + \cdots + x_n}{n} $$
    $$
    G(x_1, \ldots, x_n) = \sqrt[n]{x_1 \cdots x_n} $$
    $$
    H(x_1, \ldots, x_n) = \frac{n}{\frac{1}{x_1} + \cdots + \frac{1}{x_n}}
    $$

    - Realizar una función que calcule la mediana de una lista de números (el argumento en este caso es del tipo `list`). La mediana se define como el valor para el cual la mitad de los valores de la lista es menor que ella. Si el número de elementos es par, se toma el promedio entre los dos adyacentes.

    Realizar los cálculos para las listas de números:

    ```python
    L1 = [6.41, 1.28, 11.54, 5.13, 8.97, 3.84, 10.26, 14.1, 12.82, 16.67, 2.56, 17.95, 7.69, 15.39]
    L2 = [4.79, 1.59, 2.13, 4.26, 3.72, 1.06, 6.92, 3.19, 5.32, 2.66, 5.85, 6.39, 0.53]
    ```

   - La *moda* se define como el valor que ocurre más frecuentemente en una colección. Note que la moda puede no ser única. En ese caso debe obtener todos los valores. Escriba una función que retorne la moda de una lista de números. Utilícela para calcular la moda de la siguiente lista de números enteros:
   ```python
   L = [8, 9, 10, 11, 10, 6, 10, 17, 8, 8, 5, 10, 14, 7, 9, 12, 8, 17, 10, 12, 9, 11, 9, 12, 11, 11, 6, 9, 12, 5, 12, 9, 10, 16, 8, 4, 5, 8, 11, 12]
   ```
   
-----


.